In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive_io.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [5]:
model_name = 'rgcn_transductive_io'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [8]:
model = RGCN(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameter

Number of parameters: 3850896
Space occupied: 15403584 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory rgcn_transductive_io created successfully!


In [10]:
learning_rate = 5e-2 #1e-3 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
#         callbacks="evaluation",
#         callback_kwargs=dict(
#             evaluator=valid_evaluator,
#             evaluation_triples=dataset.validation.mapped_triples,
#             prefix="validation",
#             frequency=1,
#             additional_filter_triples=dataset.training.mapped_triples,
#         ),
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Using RGCN without graph-based sampling! Please select sampler="schlichtkrull" instead of None.
Using RGCN without graph-based sampling! Please select sampler="schlichtkrull" instead of None.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedd

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 4.502331733703613
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0006292411715911633
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0013433481062344649
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0009862946389128141
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0006292411126196384
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0013433480635285378
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.000986294588074088
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0006292410708453909
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0013433481049831544
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0009862945879142726
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validation.head.r

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

...

Step: 21
Metric: loss = 2.6677510738372803
Step: 21
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0011315093437462974
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.002973357849762914
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0020524335967546056
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0011315091978758574
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0029733579140156503
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.002052433555945754
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0011315091339196086
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0029733577513850707
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00205243344265234
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 9.731413001167769e-05
Metric: validation.both.optimistic.hits_at_1 = 4.865706500

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 22
Metric: loss = 2.5492794513702393
Step: 22
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.004415044585850374
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.004712604345117048
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.004563824465483711
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0044150445610284805
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0047126044519245625
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.004563824739307165
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.004415044471099558
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0047126042967156135
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.004563824383907586
Metric: validation.head.optimistic.hits_at_1 = 0.00126508369015181
Metric: validation.tail.optimistic.hits_at_1 = 0.0008758271701050993
Metric: validation.both.optimistic.hits_at_1

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 23
Metric: loss = 2.534865140914917
Step: 23
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.010327841560704395
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0063888461199222675
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.00835834384031333
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.010327841155230999
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.006388845853507519
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.008358344435691833
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.010327841153180086
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00638884580235615
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.008358343477768117
Metric: validation.head.optimistic.hits_at_1 = 0.0036979369404437526
Metric: validation.tail.optimistic.hits_at_1 = 0.0013623978201634877
Metric: validation.both.optimistic.hits_at_1 = 

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
           0
0   4.502332
1   4.650960
2   3.450377
3   3.892035
4   3.908746
5   3.820543
6   3.690277
7   3.576973
8   3.345960
9   3.084280
10  3.303186
11  3.266501
12  3.033059
13  2.923437
14  2.984078
15  2.990893
16  2.848908
17  2.775810
18  2.694544
19  2.702523
20  2.667751
21  2.549279
22  2.534865


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/41.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.014483   0.014483     0.014483
hits_at_1                     0.004047   0.004047     0.004047
hits_at_3                     0.012357   0.012357     0.012357
hits_at_5                     0.022305   0.022305     0.022305
hits_at_10                    0.030976   0.030976     0.030976
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.007609   0.007609     0.007609
hits_at_1                     0.001566   0.001566     0.001566
hits_at_3                     0.005709   0.005709     0.005709
hits_at_5                     0.007876   0.007876     0.007876
hits_at_10                    0.018330   0.018330     0.018330
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.011046   0.011046     0.011046
hits_at_1                     0.002806   0.002806     0.002806
hits_at_3                     0.009033  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/10.3k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.023160   0.023160     0.023160
hits_at_1                     0.007104   0.007104     0.007104
hits_at_3                     0.020728   0.020728     0.020728
hits_at_5                     0.035714   0.035714     0.035714
hits_at_10                    0.048754   0.048754     0.048754
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.010583   0.010583     0.010583
hits_at_1                     0.003309   0.003309     0.003309
hits_at_3                     0.006812   0.006812     0.006812
hits_at_5                     0.009050   0.009050     0.009050
hits_at_10                    0.024329   0.024329     0.024329
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.016872   0.016872     0.016872
hits_at_1                     0.005206   0.005206     0.005206
hits_at_3                     0.013770  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/13.1k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.022372   0.022372     0.022372
hits_at_1                     0.006576   0.006576     0.006576
hits_at_3                     0.020263   0.020263     0.020263
hits_at_5                     0.033491   0.033491     0.033491
hits_at_10                    0.048249   0.048249     0.048249
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.010761   0.010761     0.010761
hits_at_1                     0.003135   0.003135     0.003135
hits_at_3                     0.007952   0.007952     0.007952
hits_at_5                     0.010476   0.010476     0.010476
hits_at_10                    0.025998   0.025998     0.025998
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.016566   0.016566     0.016566
hits_at_1                     0.004855   0.004855     0.004855
hits_at_3                     0.014108  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   

                                                value  
0                                              cuda:0  
1                                            154035

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)